# Lab 1: Building your own n-gram language model

In this lab, you will build your own language model based on n-grams and the Maximum Likelihood Estimation.

We'll assume that our text is already "tokenized" (split up into words). We'll cover this process in more depth in the next module.

As an example, let's work with two sentences from "[The Disappearance of Lady Frances Carfax](https://en.wikipedia.org/wiki/The_Disappearance_of_Lady_Frances_Carfax)", a short story written by [Sir Arthur Conan Doyle](https://en.wikipedia.org/wiki/Arthur_Conan_Doyle).

In [15]:
# Tokens for the sentence "It shows, my dear Watson, that we are dealing
# with an exceptionally astude and dangerous man."
sample1 = ['It', 'shows', ',', 'my', 'dear', 'Watson', ',', 'that',
           'we', 'are', 'dealing', 'with', 'an', 'exceptionally',
           'astute', 'and', 'dangerous', 'man', '.']
# Tokens for the sentence "How would Lausanne do, my dear Watson?"
sample2 = ['How', 'would', 'Lausanne', 'do', ',', 'my', 'dear',
           'Watson', '?']

Your first task is to write a function that splits the `tokens` sequence
into its `n`-grams.

For instance, when `tokens=sample1` and `n=3`, your function should
return:

```python
[('It', 'shows', ','),
 ('shows', ',', 'my'),
 (',', 'my', 'dear'),
 ...,
 ('dangerous', 'man', '.')]
```
 
Note: You should return a python [`list`](https://docs.python.org/3/tutorial/datastructures.html#more-on-lists) containing [`tuple`](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences)s. `tuple`s are immutable sequences, which will be useful later on when you build your language model.

In [16]:
from typing import List, Tuple
def build_ngrams(tokens: List[str], n: int) -> List[Tuple[str]]:
    # your code here
    ngrams = []
    for i in range(len(tokens)-n+1):
        ngram = tuple(tokens[i:i+n])
        ngrams.append(ngram)
    return ngrams, len(ngrams)

# Example:
build_ngrams(sample2, n=2)




([('How', 'would'),
  ('would', 'Lausanne'),
  ('Lausanne', 'do'),
  ('do', ','),
  (',', 'my'),
  ('my', 'dear'),
  ('dear', 'Watson'),
  ('Watson', '?')],
 8)

In [17]:
# Tests:
assert len(build_ngrams(sample1, n=3)) == 17
assert build_ngrams(sample1, n=3)[0] == ('It', 'shows', ',')
assert build_ngrams(sample1, n=3)[10] == ('dealing', 'with', 'an')
assert len(build_ngrams(sample1, n=2)) == 18
assert build_ngrams(sample1, n=2)[0] == ('It', 'shows')
assert build_ngrams(sample1, n=2)[10] == ('dealing', 'with')
assert len(build_ngrams(sample2, n=2)) == 8
assert build_ngrams(sample2, n=2)[0] == ('How', 'would')
assert build_ngrams(sample2, n=2)[7] == ('Watson', '?')

AssertionError: 

With the current function, there's no way to know whether an n-gram is at the beginning, middle, or end of the sequence. To overcome this problem, n-gram language models often include special "beginning-of-string" (BOS) and "end-of-string" (EOS) control tokens.

Write a new version of your `build_ngrams` function that includes these control tokens. For instance, when `tokens=sample1` and `n=3`, your new function should return:

```python
[('<BOS>', '<BOS>', 'It'),
 ('<BOS>', 'It', 'shows'),
 ('It', 'shows', ','),
 ('shows', ',', 'my'),
 (',', 'my', 'dear'),
 ...,
 ('dangerous', 'man', '.'),
 ('man', '.', '<EOS>'),
 ('.', '<EOS>', '<EOS>')]
```

In [33]:
BOS = '<BOS>'
EOS = '<EOS>'

def build_ngrams_ctrl(tokens: List[str], n: int) -> List[Tuple[str]]:
    # your code here
    tokens = [BOS] * (n-1) + tokens + [EOS] * (n-1)
    ngrams=[]
    
    for i in range (len(tokens)-n+1):
        ngram = tuple(tokens[i:i+n])
        ngrams.append(ngram)
    return ngrams

# Example:
build_ngrams_ctrl(sample1, n=3)



[('<BOS>', '<BOS>', 'It'),
 ('<BOS>', 'It', 'shows'),
 ('It', 'shows', ','),
 ('shows', ',', 'my'),
 (',', 'my', 'dear'),
 ('my', 'dear', 'Watson'),
 ('dear', 'Watson', ','),
 ('Watson', ',', 'that'),
 (',', 'that', 'we'),
 ('that', 'we', 'are'),
 ('we', 'are', 'dealing'),
 ('are', 'dealing', 'with'),
 ('dealing', 'with', 'an'),
 ('with', 'an', 'exceptionally'),
 ('an', 'exceptionally', 'astute'),
 ('exceptionally', 'astute', 'and'),
 ('astute', 'and', 'dangerous'),
 ('and', 'dangerous', 'man'),
 ('dangerous', 'man', '.'),
 ('man', '.', '<EOS>'),
 ('.', '<EOS>', '<EOS>')]

In [ ]:
# Tests:
assert len(build_ngrams_ctrl(sample1, n=3)) == 21
assert build_ngrams_ctrl(sample1, n=3)[0] == ('<BOS>', '<BOS>', 'It')
assert build_ngrams_ctrl(sample1, n=3)[10] == ('we', 'are', 'dealing')
assert len(build_ngrams_ctrl(sample1, n=2)) == 20
assert build_ngrams_ctrl(sample1, n=2)[0] == ('<BOS>', 'It')
assert build_ngrams_ctrl(sample1, n=2)[10] == ('are', 'dealing')
assert len(build_ngrams_ctrl(sample2, n=2)) == 10
assert build_ngrams_ctrl(sample2, n=2)[0] == ('<BOS>', 'How')
assert build_ngrams_ctrl(sample2, n=2)[9] == ('?', '<EOS>')

Now that you can build n-grams, we have almost everything we need to build an n-gram language model.

To compute Maximum Likelihood Estimations, you first need to count the number of times each word follows an n-gram of size `n-1`. You can build this structure as a Python [`dict`](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) that maps the n-grams of size `n-1` to another `dict` that maps the following words to their respective counts.

For instance, when `texts=[sample1, sample2]` and `n=3`, your function should return:

```python
{
    ('<BOS>', '<BOS>'): {'It': 1, 'How': 1},
    ('<BOS>', 'It'): {'shows': 1},
    ('<BOS>', 'How'): {'would': 1},
    ...
    ('my', 'dear'): {'Watson': 2},
    ('dear', 'Watson'): {',': 1, '?': 1},
    ...
}
```

In [38]:
from typing import Dict
def count_ngrams(texts: List[List[str]], n: int) -> Dict[Tuple[str, ...], Dict[str, int]]:
    # your code here
    # Be sure to use your build_ngrams_ctrl implementation
    ngram_counts = {}

    for text_num, text in enumerate(texts, start=1):
        print(f"Procesando texto {text_num}...")
        ngrams = build_ngrams_ctrl(text, n)
        print(f"Generados {len(ngrams)} n-gramas de tamaño {n}")
        
        for ngram_num, ngram in enumerate(ngrams, start=1):
            prefix = tuple(ngram[:-1])  # n-1 gram
            word = ngram[-1]  # Última palabra en el n-grama
            

            if prefix not in ngram_counts:
                ngram_counts[prefix] = {}
            if word not in ngram_counts[prefix]:
                ngram_counts[prefix][word] = 1
            else:
                ngram_counts[prefix][word] += 1
                
    return ngram_counts

# Example:
count_ngrams([sample1, sample2], n=3)

Procesando texto 1...
Generados 21 n-gramas de tamaño 3
Procesando texto 2...
Generados 11 n-gramas de tamaño 3


{('<BOS>', '<BOS>'): {'It': 1, 'How': 1},
 ('<BOS>', 'It'): {'shows': 1},
 ('It', 'shows'): {',': 1},
 ('shows', ','): {'my': 1},
 (',', 'my'): {'dear': 2},
 ('my', 'dear'): {'Watson': 2},
 ('dear', 'Watson'): {',': 1, '?': 1},
 ('Watson', ','): {'that': 1},
 (',', 'that'): {'we': 1},
 ('that', 'we'): {'are': 1},
 ('we', 'are'): {'dealing': 1},
 ('are', 'dealing'): {'with': 1},
 ('dealing', 'with'): {'an': 1},
 ('with', 'an'): {'exceptionally': 1},
 ('an', 'exceptionally'): {'astute': 1},
 ('exceptionally', 'astute'): {'and': 1},
 ('astute', 'and'): {'dangerous': 1},
 ('and', 'dangerous'): {'man': 1},
 ('dangerous', 'man'): {'.': 1},
 ('man', '.'): {'<EOS>': 1},
 ('.', '<EOS>'): {'<EOS>': 1},
 ('<BOS>', 'How'): {'would': 1},
 ('How', 'would'): {'Lausanne': 1},
 ('would', 'Lausanne'): {'do': 1},
 ('Lausanne', 'do'): {',': 1},
 ('do', ','): {'my': 1},
 ('Watson', '?'): {'<EOS>': 1},
 ('?', '<EOS>'): {'<EOS>': 1}}

Como se puede notar, el numero de engramas no coincide con el numero de entradas del diccionario. Aquí pueden pasar dos cosas: 

1. Que ambas entradas compartan un mismo n-grama, con la misma palabra final como en:('my', 'dear'): {'Watson': 2}
2. Que para un engrama, el mismo n-grama venga seguido de palabras distintas:('dear', 'Watson'): {',': 1, '?': 1}

In [ ]:
# Tests:
assert len(count_ngrams([sample1, sample2], n=3)) == 28
assert len(count_ngrams([sample1, sample2], n=3)['<BOS>', '<BOS>']) == 2
assert count_ngrams([sample1, sample2], n=3)['<BOS>', '<BOS>']['It'] == 1
assert count_ngrams([sample1, sample2], n=3)['<BOS>', '<BOS>']['How'] == 1
assert count_ngrams([sample1, sample2], n=3)['my', 'dear']['Watson'] == 2
assert len(count_ngrams([sample1, sample2], n=2)) == 24
assert len(count_ngrams([sample1, sample2], n=2)['<BOS>',]) == 2
assert count_ngrams([sample1, sample2], n=2)['<BOS>',]['It'] == 1
assert count_ngrams([sample1, sample2], n=2)['<BOS>',]['How'] == 1
assert count_ngrams([sample1, sample2], n=2)['dear',]['Watson'] == 2

You're almost there! The last step is to convert the counts into probability estimates.

When `texts=[sample1, sample2]` and `n=3`, your function should return:

```python
{
    ('<BOS>', '<BOS>'): {'It': 0.5, 'How': 0.5},
    ('<BOS>', 'It'): {'shows': 1.0},
    ('<BOS>', 'How'): {'would': 1.0},
    ...
    ('my', 'dear'): {'Watson': 1.0},
    ('dear', 'Watson'): {',': 0.5, '?': 0.5},
    ...
}
```

In [39]:
from typing import Dict
def build_ngram_model(texts: List[List[str]], n: int) -> Dict[Tuple[str, ...], Dict[str, float]]:
    
    # your code here
    # Be sure to use your count_ngrams implementation
    ngram_counts = count_ngrams(texts, n)
    ngram_model = {}

    for prefix, word_counts in ngram_counts.items():
        total_count = sum(word_counts.values())
        probabilities = {word: count / total_count for word, count in word_counts.items()}
        ngram_model[prefix] = probabilities

    return ngram_model


# Example:
build_ngram_model([sample1, sample2], n=3)

Procesando texto 1...
Generados 21 n-gramas de tamaño 3
Procesando texto 2...
Generados 11 n-gramas de tamaño 3


{('<BOS>', '<BOS>'): {'It': 0.5, 'How': 0.5},
 ('<BOS>', 'It'): {'shows': 1.0},
 ('It', 'shows'): {',': 1.0},
 ('shows', ','): {'my': 1.0},
 (',', 'my'): {'dear': 1.0},
 ('my', 'dear'): {'Watson': 1.0},
 ('dear', 'Watson'): {',': 0.5, '?': 0.5},
 ('Watson', ','): {'that': 1.0},
 (',', 'that'): {'we': 1.0},
 ('that', 'we'): {'are': 1.0},
 ('we', 'are'): {'dealing': 1.0},
 ('are', 'dealing'): {'with': 1.0},
 ('dealing', 'with'): {'an': 1.0},
 ('with', 'an'): {'exceptionally': 1.0},
 ('an', 'exceptionally'): {'astute': 1.0},
 ('exceptionally', 'astute'): {'and': 1.0},
 ('astute', 'and'): {'dangerous': 1.0},
 ('and', 'dangerous'): {'man': 1.0},
 ('dangerous', 'man'): {'.': 1.0},
 ('man', '.'): {'<EOS>': 1.0},
 ('.', '<EOS>'): {'<EOS>': 1.0},
 ('<BOS>', 'How'): {'would': 1.0},
 ('How', 'would'): {'Lausanne': 1.0},
 ('would', 'Lausanne'): {'do': 1.0},
 ('Lausanne', 'do'): {',': 1.0},
 ('do', ','): {'my': 1.0},
 ('Watson', '?'): {'<EOS>': 1.0},
 ('?', '<EOS>'): {'<EOS>': 1.0}}

In [ ]:
# Tests:
assert build_ngram_model([sample1, sample2], n=3)['<BOS>', '<BOS>']['It'] == 0.5
assert build_ngram_model([sample1, sample2], n=3)['<BOS>', '<BOS>']['How'] == 0.5
assert build_ngram_model([sample1, sample2], n=3)['my', 'dear']['Watson'] == 1.0
assert build_ngram_model([sample1, sample2], n=2)['<BOS>',]['It'] == 0.5
assert build_ngram_model([sample1, sample2], n=2)['<BOS>',]['How'] == 0.5
assert build_ngram_model([sample1, sample2], n=2)['dear',]['Watson'] == 1.0

A language model built from only a few sentences is not very informative. Let's scale up and see what your language model looks like when we train on the complete works of Sir Arthur Conon Doyle!

In [40]:
full_text = []
with open('arthur-conan-doyle.tok.train.txt', 'rt') as fin:
    for line in fin:
        full_text.append(list(line.split()))
model = build_ngram_model(full_text, n=3)

Procesando texto 1...
Generados 7 n-gramas de tamaño 3
Procesando texto 2...
Generados 15 n-gramas de tamaño 3
Procesando texto 3...
Generados 12 n-gramas de tamaño 3
Procesando texto 4...
Generados 99 n-gramas de tamaño 3
Procesando texto 5...
Generados 35 n-gramas de tamaño 3
Procesando texto 6...
Generados 11 n-gramas de tamaño 3
Procesando texto 7...
Generados 10 n-gramas de tamaño 3
Procesando texto 8...
Generados 125 n-gramas de tamaño 3
Procesando texto 9...
Generados 12 n-gramas de tamaño 3
Procesando texto 10...
Generados 8 n-gramas de tamaño 3
Procesando texto 11...
Generados 17 n-gramas de tamaño 3
Procesando texto 12...
Generados 14 n-gramas de tamaño 3
Procesando texto 13...
Generados 11 n-gramas de tamaño 3
Procesando texto 14...
Generados 27 n-gramas de tamaño 3
Procesando texto 15...
Generados 9 n-gramas de tamaño 3
Procesando texto 16...
Generados 107 n-gramas de tamaño 3
Procesando texto 17...
Generados 113 n-gramas de tamaño 3
Procesando texto 18...
Generados 58 n-gr

Procesando texto 688...
Generados 47 n-gramas de tamaño 3
Procesando texto 689...
Generados 7 n-gramas de tamaño 3
Procesando texto 690...
Generados 8 n-gramas de tamaño 3
Procesando texto 691...
Generados 9 n-gramas de tamaño 3
Procesando texto 692...
Generados 12 n-gramas de tamaño 3
Procesando texto 693...
Generados 38 n-gramas de tamaño 3
Procesando texto 694...
Generados 91 n-gramas de tamaño 3
Procesando texto 695...
Generados 8 n-gramas de tamaño 3
Procesando texto 696...
Generados 60 n-gramas de tamaño 3
Procesando texto 697...
Generados 10 n-gramas de tamaño 3
Procesando texto 698...
Generados 8 n-gramas de tamaño 3
Procesando texto 699...
Generados 9 n-gramas de tamaño 3
Procesando texto 700...
Generados 69 n-gramas de tamaño 3
Procesando texto 701...
Generados 14 n-gramas de tamaño 3
Procesando texto 702...
Generados 100 n-gramas de tamaño 3
Procesando texto 703...
Generados 8 n-gramas de tamaño 3
Procesando texto 704...
Generados 16 n-gramas de tamaño 3
Procesando texto 705

Procesando texto 1189...
Generados 10 n-gramas de tamaño 3
Procesando texto 1190...
Generados 17 n-gramas de tamaño 3
Procesando texto 1191...
Generados 11 n-gramas de tamaño 3
Procesando texto 1192...
Generados 24 n-gramas de tamaño 3
Procesando texto 1193...
Generados 15 n-gramas de tamaño 3
Procesando texto 1194...
Generados 28 n-gramas de tamaño 3
Procesando texto 1195...
Generados 14 n-gramas de tamaño 3
Procesando texto 1196...
Generados 47 n-gramas de tamaño 3
Procesando texto 1197...
Generados 8 n-gramas de tamaño 3
Procesando texto 1198...
Generados 144 n-gramas de tamaño 3
Procesando texto 1199...
Generados 85 n-gramas de tamaño 3
Procesando texto 1200...
Generados 52 n-gramas de tamaño 3
Procesando texto 1201...
Generados 25 n-gramas de tamaño 3
Procesando texto 1202...
Generados 6 n-gramas de tamaño 3
Procesando texto 1203...
Generados 34 n-gramas de tamaño 3
Procesando texto 1204...
Generados 22 n-gramas de tamaño 3
Procesando texto 1205...
Generados 6 n-gramas de tamaño 3

Generados 61 n-gramas de tamaño 3
Procesando texto 1847...
Generados 8 n-gramas de tamaño 3
Procesando texto 1848...
Generados 31 n-gramas de tamaño 3
Procesando texto 1849...
Generados 10 n-gramas de tamaño 3
Procesando texto 1850...
Generados 15 n-gramas de tamaño 3
Procesando texto 1851...
Generados 7 n-gramas de tamaño 3
Procesando texto 1852...
Generados 10 n-gramas de tamaño 3
Procesando texto 1853...
Generados 14 n-gramas de tamaño 3
Procesando texto 1854...
Generados 86 n-gramas de tamaño 3
Procesando texto 1855...
Generados 9 n-gramas de tamaño 3
Procesando texto 1856...
Generados 74 n-gramas de tamaño 3
Procesando texto 1857...
Generados 10 n-gramas de tamaño 3
Procesando texto 1858...
Generados 79 n-gramas de tamaño 3
Procesando texto 1859...
Generados 57 n-gramas de tamaño 3
Procesando texto 1860...
Generados 21 n-gramas de tamaño 3
Procesando texto 1861...
Generados 38 n-gramas de tamaño 3
Procesando texto 1862...
Generados 18 n-gramas de tamaño 3
Procesando texto 1863...


Procesando texto 2499...
Generados 135 n-gramas de tamaño 3
Procesando texto 2500...
Generados 32 n-gramas de tamaño 3
Procesando texto 2501...
Generados 9 n-gramas de tamaño 3
Procesando texto 2502...
Generados 22 n-gramas de tamaño 3
Procesando texto 2503...
Generados 6 n-gramas de tamaño 3
Procesando texto 2504...
Generados 72 n-gramas de tamaño 3
Procesando texto 2505...
Generados 45 n-gramas de tamaño 3
Procesando texto 2506...
Generados 14 n-gramas de tamaño 3
Procesando texto 2507...
Generados 10 n-gramas de tamaño 3
Procesando texto 2508...
Generados 54 n-gramas de tamaño 3
Procesando texto 2509...
Generados 23 n-gramas de tamaño 3
Procesando texto 2510...
Generados 35 n-gramas de tamaño 3
Procesando texto 2511...
Generados 70 n-gramas de tamaño 3
Procesando texto 2512...
Generados 130 n-gramas de tamaño 3
Procesando texto 2513...
Generados 57 n-gramas de tamaño 3
Procesando texto 2514...
Generados 62 n-gramas de tamaño 3
Procesando texto 2515...
Generados 24 n-gramas de tamaño

...

Procesando texto 6845...
Generados 83 n-gramas de tamaño 3
Procesando texto 6846...
Generados 63 n-gramas de tamaño 3
Procesando texto 6847...
Generados 12 n-gramas de tamaño 3
Procesando texto 6848...
Generados 113 n-gramas de tamaño 3
Procesando texto 6849...
Generados 69 n-gramas de tamaño 3
Procesando texto 6850...
Generados 16 n-gramas de tamaño 3
Procesando texto 6851...
Generados 29 n-gramas de tamaño 3
Procesando texto 6852...
Generados 28 n-gramas de tamaño 3
Procesando texto 6853...
Generados 30 n-gramas de tamaño 3
Procesando texto 6854...
Generados 75 n-gramas de tamaño 3
Procesando texto 6855...
Generados 37 n-gramas de tamaño 3
Procesando texto 6856...
Generados 41 n-gramas de tamaño 3
Procesando texto 6857...
Generados 61 n-gramas de tamaño 3
Procesando texto 6858...
Generados 58 n-gramas de tamaño 3
Procesando texto 6859...
Generados 29 n-gramas de tamaño 3
Procesando texto 6860...
Generados 33 n-gramas de tamaño 3
Procesando texto 6861...
Generados 50 n-gramas de tamañ

Procesando texto 7522...
Generados 7 n-gramas de tamaño 3
Procesando texto 7523...
Generados 66 n-gramas de tamaño 3
Procesando texto 7524...
Generados 9 n-gramas de tamaño 3
Procesando texto 7525...
Generados 22 n-gramas de tamaño 3
Procesando texto 7526...
Generados 94 n-gramas de tamaño 3
Procesando texto 7527...
Generados 20 n-gramas de tamaño 3
Procesando texto 7528...
Generados 96 n-gramas de tamaño 3
Procesando texto 7529...
Generados 10 n-gramas de tamaño 3
Procesando texto 7530...
Generados 35 n-gramas de tamaño 3
Procesando texto 7531...
Generados 46 n-gramas de tamaño 3
Procesando texto 7532...
Generados 13 n-gramas de tamaño 3
Procesando texto 7533...
Generados 149 n-gramas de tamaño 3
Procesando texto 7534...
Generados 28 n-gramas de tamaño 3
Procesando texto 7535...
Generados 9 n-gramas de tamaño 3
Procesando texto 7536...
Generados 58 n-gramas de tamaño 3
Procesando texto 7537...
Generados 10 n-gramas de tamaño 3
Procesando texto 7538...
Generados 38 n-gramas de tamaño 3

Procesando texto 8125...
Generados 18 n-gramas de tamaño 3
Procesando texto 8126...
Generados 254 n-gramas de tamaño 3
Procesando texto 8127...
Generados 62 n-gramas de tamaño 3
Procesando texto 8128...
Generados 57 n-gramas de tamaño 3
Procesando texto 8129...
Generados 293 n-gramas de tamaño 3
Procesando texto 8130...
Generados 26 n-gramas de tamaño 3
Procesando texto 8131...
Generados 52 n-gramas de tamaño 3
Procesando texto 8132...
Generados 542 n-gramas de tamaño 3
Procesando texto 8133...
Generados 10 n-gramas de tamaño 3
Procesando texto 8134...
Generados 60 n-gramas de tamaño 3
Procesando texto 8135...
Generados 23 n-gramas de tamaño 3
Procesando texto 8136...
Generados 88 n-gramas de tamaño 3
Procesando texto 8137...
Generados 185 n-gramas de tamaño 3
Procesando texto 8138...
Generados 74 n-gramas de tamaño 3
Procesando texto 8139...
Generados 8 n-gramas de tamaño 3
Procesando texto 8140...
Generados 6 n-gramas de tamaño 3
Procesando texto 8141...
Generados 54 n-gramas de tama

Procesando texto 8676...
Generados 36 n-gramas de tamaño 3
Procesando texto 8677...
Generados 64 n-gramas de tamaño 3
Procesando texto 8678...
Generados 18 n-gramas de tamaño 3
Procesando texto 8679...
Generados 30 n-gramas de tamaño 3
Procesando texto 8680...
Generados 9 n-gramas de tamaño 3
Procesando texto 8681...
Generados 113 n-gramas de tamaño 3
Procesando texto 8682...
Generados 41 n-gramas de tamaño 3
Procesando texto 8683...
Generados 122 n-gramas de tamaño 3
Procesando texto 8684...
Generados 60 n-gramas de tamaño 3
Procesando texto 8685...
Generados 52 n-gramas de tamaño 3
Procesando texto 8686...
Generados 16 n-gramas de tamaño 3
Procesando texto 8687...
Generados 51 n-gramas de tamaño 3
Procesando texto 8688...
Generados 10 n-gramas de tamaño 3
Procesando texto 8689...
Generados 138 n-gramas de tamaño 3
Procesando texto 8690...
Generados 10 n-gramas de tamaño 3
Procesando texto 8691...
Generados 49 n-gramas de tamaño 3
Procesando texto 8692...
Generados 122 n-gramas de tam

Procesando texto 9286...
Generados 25 n-gramas de tamaño 3
Procesando texto 9287...
Generados 32 n-gramas de tamaño 3
Procesando texto 9288...
Generados 31 n-gramas de tamaño 3
Procesando texto 9289...
Generados 10 n-gramas de tamaño 3
Procesando texto 9290...
Generados 54 n-gramas de tamaño 3
Procesando texto 9291...
Generados 56 n-gramas de tamaño 3
Procesando texto 9292...
Generados 37 n-gramas de tamaño 3
Procesando texto 9293...
Generados 64 n-gramas de tamaño 3
Procesando texto 9294...
Generados 37 n-gramas de tamaño 3
Procesando texto 9295...
Generados 50 n-gramas de tamaño 3
Procesando texto 9296...
Generados 50 n-gramas de tamaño 3
Procesando texto 9297...
Generados 115 n-gramas de tamaño 3
Procesando texto 9298...
Generados 193 n-gramas de tamaño 3
Procesando texto 9299...
Generados 20 n-gramas de tamaño 3
Procesando texto 9300...
Generados 65 n-gramas de tamaño 3
Procesando texto 9301...
Generados 7 n-gramas de tamaño 3
Procesando texto 9302...
Generados 21 n-gramas de tamañ

In [41]:
for prefix in [(BOS, BOS), (BOS, 'It'), ('It', 'was'), ('my', 'dear')]:
    print(*prefix)
    sorted_probs = sorted(model[prefix].items(), key=lambda x: -x[1])
    for k, v in sorted_probs[:5]:
        print(f'\t{k}\t{v:.4f}')
    print(f'\t[{len(sorted_probs)-5} more...]')

<BOS> <BOS>
	"	0.8073
	The	0.0304
	Holmes	0.0230
	I	0.0167
	It	0.0141
	[206 more...]
<BOS> It
	was	0.8235
	is	0.0515
	may	0.0221
	had	0.0147
	seemed	0.0074
	[11 more...]
It was
	a	0.1888
	the	0.0686
	not	0.0562
	only	0.0312
	an	0.0250
	[184 more...]
my dear
	Watson	0.5612
	fellow	0.1429
	sir	0.0918
	young	0.0510
	Von	0.0204
	[12 more...]


Do these probabilities look reasonable? How can we systematically evaluate the quality of our model? We'll cover this in the next lesson!

## Optional Extra:
 - Smoothing techniques, such as [Laplace Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing), are often added to n-gram language models to deal with probabilities of 0. How might you modify your code to include smoothing?